In [29]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import NoSuchElementException
from webdrivermanager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import matplotlib.ticker as ticker
from urllib.request import urlopen
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
import seaborn as sns
import pandas as pd
import numpy as np
import selenium
import requests
import unittest
import time
import re
import sys

In [30]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

def all_links(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    # Setup the driver. This one uses chrome with some options and a path to the chromedriver
    driver = webdriver.Chrome()
    # implicitly_wait tells the driver to wait before throwing an exception
    driver.implicitly_wait(5)
    # driver.get(url) opens the page
    driver.get('https://collegedunia.com/usa-colleges')
    # This starts the scrolling by passing the driver and a timeout
    scroll(driver, 5)
    # Once scroll returns bs4 parsers the page_source
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # Them we close the driver as soup_a is storing the page source
    driver.close()
    
    alls = []
    
    for d in soup.findAll('div', attrs={'class':'jsx-1879893061 listing-block text-uppercase bg-white position-relative'}):

                name = d.find('h3', attrs={'class':'jsx-1879893061 text-white font-weight-bold text-md m-0'})
                location = d.find('span', attrs={'class':'jsx-1879893061 mr-1'})
                rating = d.find('span', attrs={'class':'jsx-1879893061 rating-text text-white font-weight-bold text-base d-block text-right'})
                scores = d.find('span', attrs={'class':'jsx-1879893061 d-block'})
                price = d.find('span', attrs={'class':'jsx-1879893061 d-flex justify-content-between'})

                all1=[]

                if name is not None:
                    #print(n[0]['alt'])
                    all1.append(name.text)
                else:
                    all1.append("N/A")

                if location is not None:
                    all1.append(location.text)
                else:    
                    all1.append('N/A')

                if rating is not None:
                    #print(rating.text)
                    all1.append(rating.text)
                else:
                    all1.append('N/A')

                if scores is not None:
                    #print(price.text)
                    all1.append(scores.text)
                else:
                    all1.append('N/A')     

                if price is not None:
                    #print(price.text)
                    all1.append(price.text)
                else:
                    all1.append('N/A')
                alls.append(all1)    
    return alls


In [31]:
results = []
for i in range(1):
    results.append(all_links(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['UNIVERSITY NAME','LOCATION','RATING','SCORES','PRICE'])

#cleaning data for ML and Data Engineering. 

df["UNIVERSITY NAME"] = df["UNIVERSITY NAME"].str.replace(',', '')
df['UNIVERSITY NAME'] = pd.to_numeric(df['UNIVERSITY NAME'], errors='ignore')

df["LOCATION"] = df["LOCATION"].str.replace(',', '')
df["LOCATION"] = df["LOCATION"].str.replace('germany', '')

df['RATING'] = df['RATING'].str.replace('/', '')
df['RATING'] = df['RATING'].str.replace('10', '')

df['PRICE'] = df['PRICE'].str.extract('([0-9][,.]*[0-9]*)')
#df['PRICE'] = df['PRICE'].apply(lambda x: find_number(x))

df[['GREGMAT','IELTSTOEFL']] = df.SCORES.str.split("|",expand=True)

df.to_csv('usa.csv', index=False, encoding='utf-8')

In [10]:
df = pd.read_csv("usa.csv")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'usa.csv'

In [11]:
df.head(10)

,UNIVERSITY NAME,LOCATION,RATING,SCORES,PRICE,GREGMAT,IELTSTOEFL
0,University of Freiburg Freiburg,Baden-Wurttemberg,6.2,TOEFL 92,2.9,TOEFL 92,NaN
1,University of Wuerzburg Wurzburg,Bavaria,6.2,NaN,NaN,NaN,NaN
2,Humboldt University of Berlin Berlin,Berlin,6.2,NaN,NaN,NaN,NaN
3,Technical University Munich Munich,Bavaria,6.2,GRE 600 | TOEFL 88,144.0,GRE 600,TOEFL 88
4,Ludwig Maximilian University of Munich Munich,Bavaria,6.2,IELTS 5.5,287.0,IELTS 5.5,NaN
5,Free University of Berlin Berlin,Berlin,6.1,IELTS 5.0,NaN,IELTS 5.0,NaN
6,University of Bonn Bonn,North Rhine-Westphalia,5.9,IELTS 6,666.0,IELTS 6,NaN
7,RWTH Aachen University Aachen,North Rhine-Westphalia,5.9,IELTS 6.5,17.0,IELTS 6.5,NaN
8,Karlsruhe Institute of Technology Karlsruhe,Baden-Wurttemberg,5.7,NaN,3.3,NaN,NaN
9,Heidelberg University Heidelberg,Baden-Wurttemberg,5.7,NaN,382.0,NaN,NaN


In [12]:
df.dtypes

UNIVERSITY NAME     object
LOCATION            object
RATING             float64
SCORES              object
PRICE              float64
GREGMAT             object
IELTSTOEFL          object
dtype: object

In [13]:
data = df.sort_values(["RATING"], axis=0, ascending=False)[:75]
data

,UNIVERSITY NAME,LOCATION,RATING,SCORES,PRICE,GREGMAT,IELTSTOEFL
0,University of Freiburg Freiburg,Baden-Wurttemberg,6.2,TOEFL 92,2.9,TOEFL 92,NaN
1,University of Wuerzburg Wurzburg,Bavaria,6.2,NaN,NaN,NaN,NaN
2,Humboldt University of Berlin Berlin,Berlin,6.2,NaN,NaN,NaN,NaN
3,Technical University Munich Munich,Bavaria,6.2,GRE 600 | TOEFL 88,144.0,GRE 600,TOEFL 88
4,Ludwig Maximilian University of Munich Munich,Bavaria,6.2,IELTS 5.5,287.0,IELTS 5.5,NaN
...,...,...,...,...,...,...,...
70,University of Veterinary Medicine Hannover Han...,Lower Saxony,4.0,NaN,NaN,NaN,NaN
71,Carl Benz School Karlsruhe,Baden-Wurttemberg,NaN,TOEFL 88,19.0,TOEFL 88,NaN
72,Karlsruhe School of Optics & Photonics Karlsruhe,Baden-Wurttemberg,NaN,IELTS 6.5,3.3,IELTS 6.5,NaN
73,RWTH International Academy Aachen,North Rhine-Westphalia,NaN,IELTS 5.5,10.0,IELTS 5.5,NaN


In [14]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure

Loading BokehJS ...

In [15]:
p = figure(x_range=data.iloc[:,0], plot_width=960, plot_height=600, title="HIGHEST RATED UNIVESITY:", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,0], top=data.iloc[:,2], width=0.6, fill_color="green")

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

show(p)

In [16]:
data = df.sort_values(["PRICE"], axis=0, ascending=False)[:75]
data

,UNIVERSITY NAME,LOCATION,RATING,SCORES,PRICE,GREGMAT,IELTSTOEFL
68,Carl von Ossietzky University of Oldenburg Old...,Lower Saxony,4.0,TOEFL 43,884.0,TOEFL 43,NaN
31,Goethe University Frankfurt Frankfurt am Main,Hesse,4.4,TOEFL 100,821.0,TOEFL 100,NaN
67,University of Goettingen Gottingen,Lower Saxony,4.0,TOEFL 81,803.0,TOEFL 81,NaN
69,Osnabruck University Osnabruck,Lower Saxony,4.0,NaN,786.0,NaN,NaN
6,University of Bonn Bonn,North Rhine-Westphalia,5.9,IELTS 6,666.0,IELTS 6,NaN
...,...,...,...,...,...,...,...
55,University of Augsburg Augsburg,Bavaria,4.1,NaN,NaN,NaN,NaN
56,University of Wuppertal Wuppertal,North Rhine-Westphalia,4.1,TOEFL 79,NaN,TOEFL 79,NaN
57,Martin Luther University Halle-Wittenberg Halle,Saxony,4.1,TOEFL 87,NaN,TOEFL 87,NaN
58,Hannover Medical School Hannover,Lower Saxony,4.1,IELTS 6,NaN,IELTS 6,NaN


In [17]:
p = figure(x_range=data.iloc[:,0], plot_width=960, plot_height=600, title="MOST EXPENSIVE UNIVESITY:", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,0], top=data.iloc[:,4], width=0.6, fill_color="yellow")
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

show(p)